In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate();

include("data.jl")
using Flux, Statistics
using Flux: onehot, onehotbatch, throttle, crossentropy, reset!, onecold

const epochs = 30

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


30

training data of bit strings from length 2 to 10

In [2]:
train = gendata(2000, 1:10)

2000-element Array{Tuple{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},Flux.OneHotVector},1}:
 ([true true true; false false false], [true, false])               
 ([false false … false false; true true … true true], [true, false])
 ([false true … false false; true false … true true], [false, true])
 ([false false … true true; true true … false false], [false, true])
 ([true true false false; false false true true], [true, false])    
 ([false true true; true false false], [false, true])               
 ([false true … true false; true false … false true], [false, true])
 ([false true … true true; true false … false false], [true, false])
 ([true false true; false true false], [false, true])               
 ([false true true; true false false], [false, true])               
 ⋮                                                                  
 ([true false … true true; false true … false false], [false, true])
 ([true false … true true; false true … false false], [true, false])
 ([false 

validation data of bit strings of length 10

In [3]:
val = gendata(100, 10)

scanner = LSTM(length(alphabet), 20)
encoder = Dense(20, length(alphabet))

function model(x)
    state = scanner.(x.data)[end]
    reset!(scanner)
    softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)
batch_loss(data) = mean(loss(d...) for d in data)

opt = ADAM(params(scanner, encoder))
evalcb = () -> @show batch_loss(val)

for i=1:epochs
    Flux.train!(loss, train, opt, cb=throttle(evalcb, 10))
end

batch_loss(val) = 0.6871599098689828 (tracked)
batch_loss(val) = 0.7002026310439651 (tracked)
batch_loss(val) = 0.6874719728519736 (tracked)
batch_loss(val) = 0.009798888544586878 (tracked)
batch_loss(val) = 0.001895060548922749 (tracked)
batch_loss(val) = 0.000517225228707835 (tracked)
batch_loss(val) = 0.00016453305926205405 (tracked)
batch_loss(val) = 5.7364294768629984e-5 (tracked)
batch_loss(val) = 2.2129669923218578e-5 (tracked)
batch_loss(val) = 1.0106233631218707e-5 (tracked)
batch_loss(val) = 5.772983184652725e-6 (tracked)
batch_loss(val) = 3.911234098793093e-6 (tracked)
batch_loss(val) = 2.9351815482314347e-6 (tracked)
batch_loss(val) = 2.343941333543809e-6 (tracked)
batch_loss(val) = 1.9492278471146982e-6 (tracked)
batch_loss(val) = 1.6674064694721478e-6 (tracked)
batch_loss(val) = 1.4562214619133842e-6 (tracked)
batch_loss(val) = 1.2921252775063072e-6 (tracked)
batch_loss(val) = 1.1609777029838474e-6 (tracked)
batch_loss(val) = 1.0537798727942274e-6 (tracked)
batch_loss(val

sanity test

In [4]:
tx = map(c -> onehotbatch(c, alphabet), [
    [false, true], # 01 -> 1
    [true, false], # 10 -> 1
    [false, false], # 00 -> 0
    [true, true]]) # 11 -> 0
[onecold(model(x)) - 1 for x in tx] |> println

[1, 1, 0, 0]


Try running the model on strings of length 50.

Even though the model has only been trained with
much shorter strings, it has learned the
parity function and will accurate on longer strings.

In [5]:
function t50()
    l = batch_loss(gendata(1000, 50))
    println(l)
end

t50 (generic function with 1 method)